In [ ]:
from transformers import AutoTokenizer
from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
import torch
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:

model = ChatGLMForConditionalGeneration.from_pretrained("thuglm").half().cuda()

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    target_modules=['query_key_value',],
)
model = get_peft_model(model, peft_config)

# 在这里加载lora模型，注意修改chekpoint
peft_path = "test003/checkpoint-240/chatglm-lora.pt"
model.load_state_dict(torch.load(peft_path), strict=False)
model.eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("thuglm", trust_remote_code=True)

In [ ]:
text = "给出三个保持健康的秘诀" #"生成一个人野营旅行可能需要的十件物品的清单"#

with torch.autocast("cuda"):
    res, history = model.chat(tokenizer=tokenizer, query=text,max_length=300)
    print(res)